In [1]:
# Import required libraries
from path import Path
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.preprocessing import LabelEncoder, StandardScaler
from pybaseball import batting_stats
from pybaseball import batting_stats_range
from pybaseball import pitching_stats_range
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [12]:
csv_path = Path('mlb_odds_2019.csv')
df= pd.read_csv(csv_path)
df.head()

,Date,Rot,VH,Team,Pitcher,1st,2nd,3rd,4th,5th,...,9th,Final,Open,Close,Run Line,Unnamed: 18,Open OU,Unnamed: 20,Close OU,Unnamed: 22
0,320,901,N,SEA,MGONZALES-L,0,0,5,1,3,...,0,9,107,120,1.5,-160,8.5,-110,8.5,-120
1,320,902,N,OAK,MFIERS-R,1,1,2,0,0,...,0,7,-127,-130,-1.5,140,8.5,-110,8.5,100
2,321,903,N,SEA,YKIKUCHI-L,0,2,1,0,0,...,0,5,109,-125,1.5,-262,9.0,-110,9.0,-130
3,321,904,N,OAK,MESTRADA-R,0,0,0,0,2,...,0,4,-129,115,-1.5,216,9.0,-110,9.0,110
4,328,905,V,NYM,JDEGROM-R,1,0,0,0,0,...,0,2,110,115,1.5,-215,6.5,-110,6.5,-105


In [2]:
start = datetime.strptime("2019-03-28", "%Y-%m-%d")
start = start + timedelta(7)
end = datetime.strptime("2019-09-29", "%Y-%m-%d")
date_generated = [start + timedelta(days=x) for x in range(0, (end-start).days)]

#maybe this chunk can be inside the past comprehensive list
dates = []
for date in date_generated:
    date = date.strftime("%Y-%m-%d")
    dates.append(date)
#####

date_count = 0
todays_date = []
yesterdays_date_list = []
week_prior_list = []
week_prior_list_pitch = []

for date in dates:
    date = datetime.strptime(dates[date_count], '%Y-%m-%d')
    yesterdays_date = date + timedelta(-1)
    week_prior = date + timedelta(-8)
    week_prior_pitch = date + timedelta(-11)
    date = date.strftime("%Y-%m-%d")
    week_prior_date = week_prior.strftime("%Y-%m-%d")
    week_prior_date_pitch = week_prior_pitch.strftime("%Y-%m-%d")
    yesterdays_date_date = yesterdays_date.strftime("%Y-%m-%d")
    todays_date.append(date)
    week_prior_list.append(week_prior_date)
    yesterdays_date_list.append(yesterdays_date_date)
    week_prior_list_pitch.append(week_prior_date_pitch)
    date_count += 1

In [3]:
# Define function for cleaning dataframe
def clean_lineups(df):
    positions = ['1st','2nd','3rd','4th','5th','6th','7th','8th','9th']
    pos_count = 0
    for row in positions:
        
        # New data frame with split value columns 
        new = df[positions[pos_count]].str.split("-", n = 1, expand = True) 

        # Making separate name column from new data frame 
        df[positions[pos_count]] = new[0] 

        # Making separate position column from new data frame 
        df["Position"]= new[1]
        df.drop(columns = 'Position')
        pos_count += 1

In [4]:
# Define function for cleaning Game column
def clean_game_column(df):
    new = df['Game (# = Opp. LHS)'].str.split(" ", n = 5, expand = True)
    df['Game Date'] = new[1]
    df['Win Loss'] = new[4]
    new_date = df['Game Date'].str.split('/', n = 1, expand = True)
    df['Month'] = new_date[0]
    df['Date'] = new_date[1]
    new_month = df['Month'].str[3]
    
    df['Month'] = new_month

In [5]:
date_count_1 = 0
series_list_1 = []
series_list_2 = []
series_list_3 = []
series_list_4 = []
while date_count_1 < 178:
    if date_count_1 == 44:
        date_count_1 += 1
    elif date_count_1 == 94:
        date_count_1 += 5
    # Retrieve initial dataframes from pybaseball
    batting_data = batting_stats_range(week_prior_list[date_count_1], yesterdays_date_list[date_count_1])
    pitching_data = pitching_stats_range(week_prior_list[date_count_1], yesterdays_date_list[date_count_1])
    todays_batting_data = batting_stats_range(todays_date[date_count_1])
    yesterdays_batting_data = batting_stats_range(yesterdays_date_list[date_count_1])
    
    
    # Change team names to account for teams in different leagues within the same city
    batting_data_tm = batting_data.copy()
    batting_data_tm.Tm = batting_data_tm.Tm + batting_data_tm.Lev
    todays_batting_data.Tm = todays_batting_data.Tm + todays_batting_data.Lev
    todays_batting_data = todays_batting_data.drop(columns = ['Lev', 'Opp'])
    todays_batting_data = todays_batting_data[['Name', 'Tm']].set_index(['Name', 'Tm'])
    batting_data_tm = batting_data_tm.set_index(['Name', 'Tm']).drop(columns = ['Lev', 'Age','#days'])
    concat = pd.concat([todays_batting_data, batting_data_tm],axis = 1, join = 'inner')
    concat = concat.reset_index()
    # Create dataframe for names in batter dataframe
    names = concat['Name']
    teams_batter = concat
    # Encode teams
    label_encoder = LabelEncoder()
    label_encoder.fit(teams_batter["Tm"])
    team_names_str = label_encoder.classes_
    # Transform batter data
    teams_batter["Team Int"] = label_encoder.transform(teams_batter["Tm"])
    teams_batter = teams_batter.reset_index().set_index('Team Int').drop(columns = ['index'])
    # Replace NaN's with zeroes
    teams_batter = teams_batter.replace([np.NaN], 0)
    # Teams dictionary
    team_names_data = {
        "ArizonaMLB-NL": 'D_Backs',
        "AtlantaMLB-NL": 'Braves',
        "BaltimoreMLB-AL": 'Orioles',
        "BostonMLB-AL": 'Red_Sox',
        "ChicagoMLB-NL": 'Cubs',
        "ChicagoMLB-AL": 'White_Sox',
        "CincinnatiMLB-NL": 'Reds',
        "ClevelandMLB-AL": 'Indians',
        "ColoradoMLB-NL": 'Rockies',
        "DetroitMLB-AL": 'Tigers',
        "HoustonMLB-AL": 'Astros',
        "Kansas CityMLB-AL": 'Royals',
        "Los AngelesMLB-AL": 'Angels',
        "Los AngelesMLB-NL": 'Dodgers',
        "MiamiMLB-NL": 'Marlins',
        "MilwaukeeMLB-NL": 'Brewers',
        "MinnesotaMLB-AL": 'Twins',
        "New YorkMLB-NL": 'Mets',
        "New YorkMLB-AL": 'Yankees',
        "OaklandMLB-AL": 'As',
        "PhiladelphiaMLB-NL": 'Phillies',
        "PittsburghMLB-NL": 'Pirates',
        "San DiegoMLB-NL": 'Padres',
        "SeattleMLB-AL": 'Mariners',
        "San FranciscoMLB-NL": 'Giants',
        "St. LouisMLB-NL": 'Cards',
        "Tampa BayMLB-AL": 'Rays',
        "TexasMLB-AL": 'Rangers',
        "TorontoMLB-AL": 'Jays',
        "WashingtonMLB-NL": 'Nats',
    }
    teams_batter['Tm'] = teams_batter["Tm"].apply(lambda x: team_names_data[x])
    # Split data into each team
    label_encoder = LabelEncoder()
    label_encoder.fit(teams_batter["Tm"])
    team_names_str = label_encoder.classes_
    team_count = 0
    teams_list = ['ArizonaMLB-NL','AtlantaMLB-NL', 'BaltimoreMLB-AL', 'BostonMLB-AL', 'ChicagoMLB-AL', 'ChicagoMLB-NL', 'CincinnatiMLB-NL', 'ClevelandMLB-AL', 'ColoradoMLB-NL', 'DetroitMLB-AL', 'HoustonMLB-AL', 'Kansas_CityMLB-AL', 'Los_AngelesMLB-AL', 'Los_AngelesMLB-NL', 'MiamiMLB-NL', 'MilwaukeeMLB-NL', 'MinnesotaMLB-AL', 'New_YorkMLB-AL', 'New_YorkMLB-NL', 'OaklandMLB-AL', 'PhiladelphiaMLB-NL', 'PittsburghMLB-NL', 'San_DiegoMLB-NL', 'San_FranciscoMLB-NL', 'SeattleMLB-AL', 'St_LouisMLB-NL', 'Tampa_BayMLB-AL', 'TexasMLB-AL', 'TorontoMLB-AL', 'WashingtonMLB-NL']
    stats_list = ['PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB','IBB', 'SO', 'HBP', 'SH', 'SF', 'GDP', 'SB', 'CS']
    team_df_list = []
    for name in team_names_str:
        stats_count = 0

        team_name_var = team_names_str[team_count] + '_Team'
        vars()[team_name_var] = teams_batter.groupby('Team Int').get_group(team_count).reset_index()
        team_df_list.append(teams_batter.groupby('Team Int').get_group(team_count).reset_index())
        vars()[team_name_var]['Players'] = 1
        tm = team_names_str[team_count]
        vars()[team_name_var] = vars()[team_name_var].groupby('Team Int').sum()
        vars()[team_name_var]['G'] = vars()[team_name_var]['G']/vars()[team_name_var]['Players']
        for stat in stats_list:
            vars()[team_name_var][stats_list[stats_count]] = (vars()[team_name_var][stats_list[stats_count]])/(vars()[team_name_var]['G'])
            stats_count += 1
        vars()[team_name_var]['BA'] = vars()[team_name_var]['H']/vars()[team_name_var]['AB']
        vars()[team_name_var]['OBP'] = (vars()[team_name_var]['H'] + vars()[team_name_var]['BB'] + vars()[team_name_var]['IBB'] + vars()[team_name_var]['HBP'])/vars()[team_name_var]['PA']
        vars()[team_name_var]['SLG'] = (vars()[team_name_var]['H'] + 2*(vars()[team_name_var]['2B']) + 3*(vars()[team_name_var]['3B']) + 4*(vars()[team_name_var]['HR']))/vars()[team_name_var]['AB']
        vars()[team_name_var]['OPS'] = vars()[team_name_var]['SLG'] + vars()[team_name_var]['OBP']
        vars()[team_name_var]['G'] = vars()[team_name_var]['G']/vars()[team_name_var]['G']
        vars()[team_name_var]['PA'] = vars()[team_name_var]['PA']/vars()[team_name_var]['Players']
        vars()[team_name_var]['AB'] = vars()[team_name_var]['AB']/vars()[team_name_var]['Players']
        vars()[team_name_var]['Tm'] = tm
        vars()[team_name_var] = vars()[team_name_var].set_index('Tm')
        team_count += 1
    # Read in MLB Schedule CSV
    sched_path = Path('../Baseball/MLB_schedule_2019.csv')
    schedule = pd.read_csv(sched_path)
    # Set index for dataframe
    schedule['Date'] = schedule['2019']
    schedule["Month"] = schedule['2019']
    for x in range(194):

            # New data frame with split value columns 
            new = schedule['2019'][x].split("-") 

            # Making separate name column from new data frame 
            schedule['Date'][x] = new[0] 

            # Making separate position column from new data frame 
            schedule["Month"][x] = new[1]
    # Pull in every teams starting lineups and clean them
    team_count = 0
    team_names_list = []
    for teams in team_names_str:
        csv_name = '../Baseball/Lineups/'+team_names_str[team_count]+'_Lineup.csv'
        path = Path(csv_name)
        team_name_var = team_names_str[team_count] + '_Lineups'
        vars()[team_name_var] = pd.read_csv(path)
        team_names_list.append(vars()[team_name_var])
        team_count += 1
    months = {
        "3": 'Mar',
        "4": 'Apr',
        "5": 'May',
        "6": 'Jun',
        "7": 'Jul',
        "8": 'Aug',
        "9": 'Sep',
        "10": 'Oct',
    }
    datetime_formatting = {
        'Mar': '2019-03',
        'Apr':'2019-04',
        'May':'2019-05',
        'Jun':'2019-06',
        'Jul':'2019-07',
        'Aug':'2019-08',
        'Sep':'2019-09',
        'Oct':'2019-09',
    }
    day_formatting = {
        '1': '01',
        '2':'02',
        '3':'03',
        '4':'04',
        '5':'05',
        '6':'06',
        '7':'07',
        '8':'08',
        '9':'09',
        '10': '10',
        '11':'11',
        '12':'12',
        '13':'13',
        '14':'14',
        '15':'15',
        '16':'16',
        '17':'17',
        '18':'18',
        '19': '19',
        '20':'20',
        '21':'21',
        '22':'22',
        '23':'23',
        '24':'24',
        '25':'25',
        '26':'26',
        '27':'27',
        '28':'28',
        '29':'29',
        '30':'30',
        '31':'31',
    }
    win_loss = {
        'W':1,
        'L':0,
    }
    team_count = 0
    team_lineups = []
    for teams in team_names_str:
        team_name_var = team_names_str[team_count] + '_Lineups'
        clean_lineups(vars()[team_name_var])
        clean_game_column(vars()[team_name_var])

        team_count += 1
        vars()[team_name_var]["Month"] = vars()[team_name_var]["Month"].apply(lambda x: months[x]).apply(lambda x: datetime_formatting[x])
        vars()[team_name_var]["Date"] = vars()[team_name_var]["Date"].apply(lambda x: day_formatting[x])
        vars()[team_name_var]['Game Date'] = vars()[team_name_var]['Month'] + '-' + vars()[team_name_var]['Date']
        vars()[team_name_var] = vars()[team_name_var].drop(columns = ['Month','Date', 'Game (# = Opp. LHS)','Position'])
        vars()[team_name_var] = vars()[team_name_var].set_index('Game Date')
        team_lineups.append(vars()[team_name_var])
    # Teams dictionary
    team_names_schedule = {
        "Ari": 'D_Backs',
        "Atl": 'Braves',
        "Bal": 'Orioles',
        "Bos": 'Red_Sox',
        "ChC": 'Cubs',
        "ChW": 'White_Sox',
        "Cin": 'Reds',
        "Cle": 'Indians',
        "Col": 'Rockies',
        "Det": 'Tigers',
        "Hou": 'Astros',
        "Kan": 'Royals',
        "Ana": 'Angels',
        "Los": 'Dodgers',
        "Fla": 'Marlins',
        "Mil": 'Brewers',
        "Min": 'Twins',
        "NYM": 'Mets',
        "NYY": 'Yankees',
        "Oak": 'As',
        "Phi": 'Phillies',
        "Pit": 'Pirates',
        "Sdg": 'Padres',
        "Sea": 'Mariners',
        "Sfo": 'Giants',
        "StL": 'Cards',
        "Tam": 'Rays',
        "Tex": 'Rangers',
        "Tor": 'Jays',
        "Was": 'Nats',
        "@Ari": 'D_Backs',
        "@Atl": 'Braves',
        "@Bal": 'Orioles',
        "@Bos": 'Red_Sox',
        "@ChC": 'Cubs',
        "@ChW": 'White_Sox',
        "@Cin": 'Reds',
        "@Cle": 'Indians',
        "@Col": 'Rockies',
        "@Det": 'Tigers',
        "@Hou": 'Astros',
        "@Kan": 'Royals',
        "@Ana": 'Angels',
        "@Los": 'Dodgers',
        "@Fla": 'Marlins',
        "@Mil": 'Brewers',
        "@Min": 'Twins',
        "@NYM": 'Mets',
        "@NYY": 'Yankees',
        "@Oak": 'As',
        "@Phi": 'Phillies',
        "@Pit": 'Pirates',
        "@Sdg": 'Padres',
        "@Sea": 'Mariners',
        "@Sfo": 'Giants',
        "@StL": 'Cards',
        "@Tam": 'Rays',
        "@Tex": 'Rangers',
        "@Tor": 'Jays',
        "@Was": 'Nats',
        'Nan': 'Nan',
    }
    # Clean opponent names in schedule dataframe
    schedule = schedule.replace([np.NaN], 'Nan')
    teams_list = ['ArizonaMLB-NL','AtlantaMLB-NL', 'BaltimoreMLB-AL', 'BostonMLB-AL', 'ChicagoMLB-AL', 'ChicagoMLB-NL', 'CincinnatiMLB-NL', 'ClevelandMLB-AL', 'ColoradoMLB-NL', 'DetroitMLB-AL', 'HoustonMLB-AL', 'Kansas_CityMLB-AL', 'Los_AngelesMLB-AL', 'Los_AngelesMLB-NL', 'MiamiMLB-NL', 'MilwaukeeMLB-NL', 'MinnesotaMLB-AL', 'New_YorkMLB-AL', 'New_YorkMLB-NL', 'OaklandMLB-AL', 'PhiladelphiaMLB-NL', 'PittsburghMLB-NL', 'San_DiegoMLB-NL', 'San_FranciscoMLB-NL', 'SeattleMLB-AL', 'St_LouisMLB-NL', 'Tampa_BayMLB-AL', 'TexasMLB-AL', 'TorontoMLB-AL', 'WashingtonMLB-NL']
    count = 0
    for teams in team_names_str:
        schedule[team_names_str[count]] = schedule[team_names_str[count]].apply(lambda x: team_names_schedule[x])
        count += 1
    # Convert every Month-Date into YYY:MM:DD format
    datetime_formatting = {
        'Mar': '2019-03',
        'Apr':'2019-04',
        'May':'2019-05',
        'Jun':'2019-06',
        'Jul':'2019-07',
        'Aug':'2019-08',
        'Sep':'2019-09',
        'Oct':'2019-09',
    }
    day_formatting = {
        '1': '01',
        '2':'02',
        '3':'03',
        '4':'04',
        '5':'05',
        '6':'06',
        '7':'07',
        '8':'08',
        '9':'09',
        '10': '10',
        '11':'11',
        '12':'12',
        '13':'13',
        '14':'14',
        '15':'15',
        '16':'16',
        '17':'17',
        '18':'18',
        '19': '19',
        '20':'20',
        '21':'21',
        '22':'22',
        '23':'23',
        '24':'24',
        '25':'25',
        '26':'26',
        '27':'27',
        '28':'28',
        '29':'29',
        '30':'30',
        '31':'31',
    }
    schedule['Month'] = schedule['Month'].apply(lambda x: datetime_formatting[x])
    schedule['Date'] = schedule['Date'].apply(lambda x: day_formatting[x])
    # Separate each teams' schedule
    schedule['Game Date'] = schedule['Month'] + '-' + schedule['Date']
    schedule = schedule.drop(columns = ['2019','Date','Month']).set_index('Game Date')
    teams_list = ['ArizonaMLB-NL','AtlantaMLB-NL', 'BaltimoreMLB-AL', 'BostonMLB-AL', 'ChicagoMLB-AL', 'ChicagoMLB-NL', 'CincinnatiMLB-NL', 'ClevelandMLB-AL', 'ColoradoMLB-NL', 'DetroitMLB-AL', 'HoustonMLB-AL', 'Kansas_CityMLB-AL', 'Los_AngelesMLB-AL', 'Los_AngelesMLB-NL', 'MiamiMLB-NL', 'MilwaukeeMLB-NL', 'MinnesotaMLB-AL', 'New_YorkMLB-AL', 'New_YorkMLB-NL', 'OaklandMLB-AL', 'PhiladelphiaMLB-NL', 'PittsburghMLB-NL', 'San_DiegoMLB-NL', 'San_FranciscoMLB-NL', 'SeattleMLB-AL', 'St_LouisMLB-NL', 'Tampa_BayMLB-AL', 'TexasMLB-AL', 'TorontoMLB-AL', 'WashingtonMLB-NL']
    teams_count = 0
    team_schedule = []
    for col in team_names_str:
        team_name_var = team_names_str[teams_count] + '_Schedule'
        vars()[team_name_var] = schedule[team_names_str[teams_count]]
        vars()[team_name_var] = pd.DataFrame(vars()[team_name_var])
        vars()[team_name_var]['Tm'] = team_names_str[teams_count]
        vars()[team_name_var]
        team_schedule.append(vars()[team_name_var])
        teams_count += 1
    count = 0
    for name in team_names_str:
        team_name_var =  'Schedule_Lineups_' + team_names_str[count]
        vars()[team_name_var] = pd.concat([team_schedule[count],team_lineups[count]], axis = 1, join = 'inner')
        vars()[team_name_var] = vars()[team_name_var].drop(columns = ['1st','2nd','3rd','4th','5th','6th','7th','8th','9th'])
        vars()[team_name_var] = vars()[team_name_var].reset_index()
        vars()[team_name_var] = vars()[team_name_var].groupby('Game Date').get_group(todays_date[date_count_1])
        vars()[team_name_var]["Win Loss"] = vars()[team_name_var]["Win Loss"].apply(lambda x: win_loss[x])
        vars()[team_name_var] = vars()[team_name_var].set_index('Tm')
        vars()[team_name_var].columns = ['Game Date','Opponent','Win Loss']
        count += 1
    team_stats_list = []
    count = 0
    for name in team_names_str:
        team_name_var =  'Schedule_Lineups_' + team_names_str[count]
        team_var = team_names_str[count] + '_Team'
        #print(vars()[team_name_var])
        #print(vars()[team_var])
        vars()[team_names_str[count]] = pd.concat([vars()[team_name_var], vars()[team_var]],axis = 1, join = 'inner').reset_index()  
        team_stats_list.append(vars()[team_names_str[count]])
        count += 1
    count = 0
    #print(team_stats_list)
    opponent_list = []
    for item in team_stats_list:
        #print(team_stats_list[count]['Opponent'][0])
        opponent_list.append(team_stats_list[count]['Opponent'][0])
        count += 1
    opponent_df = pd.DataFrame(opponent_list)
    team_names_df = pd.DataFrame(team_names_str)
    count_df = pd.concat([team_names_df, opponent_df], axis = 1)
    count_df.columns = ['Tm', 'Opponent']
    count = 0
    count_list_1 = []
    count_list_2 = []
    for name in team_names_str:
        team_name = count_df['Tm'][count]
        for x in range(count):
            if count_df['Opponent'][x] == team_name:
                break
            else:
                None
        if count < x:
            count_list_1.append(count)
        else:
            count_list_2.append(count)
        count += 1
    stats_list = ['PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB','IBB', 'SO', 'HBP', 'SH', 'SF', 'GDP', 'SB', 'CS', 'BA','OBP','SLG','OPS']
    stats_list_opp = ['PA_opp', 'AB_opp', 'R_opp', 'H_opp', '2B_opp', '3B_opp', 'HR_opp', 'RBI_opp', 'BB_opp','IBB_opp', 'SO_opp', 'HBP_opp', 'SH_opp', 'SF_opp', 'GDP_opp', 'SB_opp', 'CS_opp', 'BA_opp','OBP_opp','SLG_opp','OPS_opp']
    for count in count_list_1:
        # Initialize count for stats
        stats_count = 0
        # Establish team name variables to call in previous df's
        team_name_var = team_names_str[count]
        # Create a new variable concatinated df name
        concat_name = team_names_str[count] + '_Concat'
        # Create a new opponent vaiable name
        opp_name_var = opponent_list[count]
        # Set index as 'Opponent' for team name variable
        vars()[team_name_var] = vars()[team_name_var].set_index('Tm')
        # Create a new variable name for opponent
        new_opp = opponent_list[count] + '_Opp'
        # print(new_opp)
        # Copy over old dataframe to new one so as to not alter the original
        vars()[new_opp] = (vars()[opp_name_var]).copy()
        # Set new df index to 'Opponent'
        vars()[new_opp] = vars()[opp_name_var].set_index('Opponent')

        vars()[new_opp].columns = ['Tm','Game Date_opp','Win Loss_opp','G_opp','PA_opp','AB_opp','R_opp','H_opp','2B_opp','3B_opp','HR_opp','RBI_opp','BB_opp','IBB_opp', 'SO_opp', 'HBP_opp', 'SH_opp', 'SF_opp', 'GDP_opp', 'SB_opp', 'CS_opp','BA_opp','OBP_opp','SLG_opp','OPS_opp','Players_opp']
        vars()[concat_name] = pd.concat([vars()[team_name_var], vars()[new_opp]], axis = 1, join = 'inner')
        for stat in stats_list:
            vars()[concat_name][stats_list[stats_count]] = vars()[concat_name][stats_list[stats_count]] - vars()[concat_name][stats_list_opp[stats_count]]
            stats_count += 1
        vars()[concat_name] = vars()[concat_name].drop(columns = ['PA_opp', 'AB_opp', 'R_opp', 'H_opp', '2B_opp', '3B_opp', 'HR_opp', 'RBI_opp', 'BB_opp','IBB_opp', 'SO_opp', 'HBP_opp', 'SH_opp', 'SF_opp', 'GDP_opp', 'SB_opp', 'CS_opp', 'BA_opp','OBP_opp','SLG_opp','OPS_opp', 'Players_opp','Players', 'Tm','Game Date_opp','G_opp','Win Loss_opp'])
        vars()[concat_name] = vars()[concat_name].reset_index()
        vars()[concat_name] = vars()[concat_name].set_index('Opponent')
        vars()[concat_name].columns = ['Tm','Game Date','Win Loss','G','PA','AB','R','H','2B','3B','HR','RBI','BB','IBB', 'SO', 'HBP', 'SH', 'SF', 'GDP', 'SB', 'CS','BA','OBP','SLG','OPS']
        series_list_2.append(vars()[concat_name])
    stats_list = ['PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB','IBB', 'SO', 'HBP', 'SH', 'SF', 'GDP', 'SB', 'CS', 'BA','OBP','SLG','OPS']
    stats_list_opp = ['PA_opp', 'AB_opp', 'R_opp', 'H_opp', '2B_opp', '3B_opp', 'HR_opp', 'RBI_opp', 'BB_opp','IBB_opp', 'SO_opp', 'HBP_opp', 'SH_opp', 'SF_opp', 'GDP_opp', 'SB_opp', 'CS_opp', 'BA_opp','OBP_opp','SLG_opp','OPS_opp', 'Win Loss_opp']
    for count in count_list_2:
        # Initialize a count for stats
        stats_count = 0
        # Pull team var names to call in old df's
        team_name_var = team_names_str[count]

        vars()[team_name_var] = vars()[team_name_var].set_index('Opponent')
        # vars()[team_name_var] = vars()[concat_name].set_index('Opponent')
        # Create a new concat variable name
        concat_name = team_names_str[count] + '_Concat'
        # Create an opponent variable name
        opp_name_var = opponent_list[count]
        # Create a new opp name to not disturb old df
        new_opp = opponent_list[count] + '_Opp'
        # Copy old df to new variable name
        vars()[new_opp] = (vars()[opp_name_var]).copy()
        if len(vars()[new_opp].columns)== 26:
            vars()[new_opp] = vars()[new_opp].reset_index()
        else:    
            None
        vars()[new_opp].columns = ['Opponent','Game Date_opp','Tm','Win Loss_opp','G_opp','PA_opp','AB_opp','R_opp','H_opp','2B_opp','3B_opp','HR_opp','RBI_opp','BB_opp','IBB_opp', 'SO_opp', 'HBP_opp', 'SH_opp', 'SF_opp', 'GDP_opp', 'SB_opp', 'CS_opp','BA_opp','OBP_opp','SLG_opp','OPS_opp','Players_opp']
        vars()[new_opp] = vars()[new_opp].set_index('Opponent')
        vars()[concat_name] = pd.concat([vars()[team_name_var], vars()[new_opp]], axis = 1, join = 'inner')
        for stat in stats_list:
            vars()[concat_name][stats_list[stats_count]] = vars()[concat_name][stats_list[stats_count]] - vars()[concat_name][stats_list_opp[stats_count]]
            stats_count += 1
        vars()[concat_name] = vars()[concat_name].drop(columns = ['PA_opp', 'AB_opp', 'R_opp', 'H_opp', '2B_opp', '3B_opp', 'HR_opp', 'RBI_opp', 'BB_opp','IBB_opp', 'SO_opp', 'HBP_opp', 'SH_opp', 'SF_opp', 'GDP_opp', 'SB_opp', 'CS_opp', 'BA_opp','OBP_opp','SLG_opp','OPS_opp', 'Players_opp','Game Date_opp','Tm','Win Loss_opp','G_opp', 'Players'])
        vars()[concat_name]['Tm'] = team_name_var
        series_list_1.append(vars()[concat_name])
    pitching_data = pitching_stats_range(week_prior_list_pitch[date_count_1], yesterdays_date_list[date_count_1])
    todays_pitching_data = pitching_stats_range(todays_date[date_count_1])
    yesterdays_pitching_data = pitching_stats_range(yesterdays_date_list[date_count_1])
    # Change team names to account for teams in different leagues within the same city
    pitching_data_tm = pitching_data.copy()
    pitching_data_tm.Tm = pitching_data_tm.Tm + pitching_data_tm.Lev
    todays_pitching_data.Tm = todays_pitching_data.Tm + todays_pitching_data.Lev
    todays_pitching_data = todays_pitching_data.drop(columns = ['Lev', 'Opp'])
    todays_pitching_data = todays_pitching_data[['Name', 'Tm']].set_index(['Name', 'Tm'])
    pitching_data_tm = pitching_data_tm.set_index(['Name', 'Tm']).drop(columns = ['Lev', 'Age','#days'])
    concat = pd.concat([todays_pitching_data, pitching_data_tm],axis = 1, join = 'inner')
    concat = concat.reset_index()
    # Create dataframe for names in batter dataframe
    names = concat['Name']
    teams_batter = concat
    # Encode teams
    label_encoder = LabelEncoder()
    label_encoder.fit(teams_batter["Tm"])
    team_names_str = label_encoder.classes_
    # Transform batter data
    teams_batter["Team Int"] = label_encoder.transform(teams_batter["Tm"])
    teams_batter = teams_batter.reset_index().set_index('Team Int').drop(columns = ['index'])
    # Replace NaN's with zeroes
    teams_batter = teams_batter.replace([np.NaN], 0)
    teams_batter = teams_batter.drop(columns = ['W','L', 'SV'])
    teams_batter['Tm'] = teams_batter["Tm"].apply(lambda x: team_names_data[x])
    stats_list = teams_batter.drop(columns = ['Name','Tm']).columns.to_list()
    stats_count = 0
    for stat in stats_list:
        stats_list[stats_count] = stats_list[stats_count] + '_Pitch'
        stats_count += 1
    stats_list.append('Players_Pitch')
    # Split data into each team
    team_count = 0
    label_encoder = LabelEncoder()
    label_encoder.fit(teams_batter["Tm"])
    team_names_str = label_encoder.classes_
    teams_list = ['ArizonaMLB-NL','AtlantaMLB-NL', 'BaltimoreMLB-AL', 'BostonMLB-AL', 'ChicagoMLB-AL', 'ChicagoMLB-NL', 'CincinnatiMLB-NL', 'ClevelandMLB-AL', 'ColoradoMLB-NL', 'DetroitMLB-AL', 'HoustonMLB-AL', 'Kansas_CityMLB-AL', 'Los_AngelesMLB-AL', 'Los_AngelesMLB-NL', 'MiamiMLB-NL', 'MilwaukeeMLB-NL', 'MinnesotaMLB-AL', 'New_YorkMLB-AL', 'New_YorkMLB-NL', 'OaklandMLB-AL', 'PhiladelphiaMLB-NL', 'PittsburghMLB-NL', 'San_DiegoMLB-NL', 'San_FranciscoMLB-NL', 'SeattleMLB-AL', 'St_LouisMLB-NL', 'Tampa_BayMLB-AL', 'TexasMLB-AL', 'TorontoMLB-AL', 'WashingtonMLB-NL']
    team_df_list = []
    for name in team_names_str:
        stat_count = 0

        team_name_var = team_names_str[team_count] + '_Team'
        team_int = team_count
        vars()[team_name_var] = teams_batter.groupby('Team Int').get_group(team_count).reset_index()
        team_df_list.append(teams_batter.groupby('Team Int').get_group(team_count).reset_index())
        vars()[team_name_var]['Players'] = 1
        tm = team_names_str[team_count]
        vars()[team_name_var] = vars()[team_name_var].groupby('Team Int').sum()
        vars()[team_name_var]['G'] = vars()[team_name_var]['G']/vars()[team_name_var]['Players']
        vars()[team_name_var].columns = stats_list
        for stat in stats_list:
            vars()[team_name_var][stats_list[stat_count]] = (vars()[team_name_var][stats_list[stat_count]])/(vars()[team_name_var]['G_Pitch'])
            stats_count += 1
        vars()[team_name_var]['WHIP_Pitch'] = (vars()[team_name_var]['BB_Pitch'] +vars()[team_name_var]['IBB_Pitch'] + vars()[team_name_var]['H_Pitch']) /vars()[team_name_var]['IP_Pitch']
        vars()[team_name_var]['G'] = vars()[team_name_var]['G_Pitch']/vars()[team_name_var]['G_Pitch']
        # BABIP={{H-HR}/{AB-K-HR+SF}}
        vars()[team_name_var]['BAbip_Pitch'] = (vars()[team_name_var]['H_Pitch'] - vars()[team_name_var]['HR_Pitch'])/(vars()[team_name_var]['AB_Pitch'] - vars()[team_name_var]['HR_Pitch'] - vars()[team_name_var]['BAbip_Pitch'] + vars()[team_name_var]['SF_Pitch'])
        vars()[team_name_var]['Tm'] = tm
        vars()[team_name_var]['Team Int'] = team_int
        vars()[team_name_var] = vars()[team_name_var].drop(columns = ('G'))
        vars()[team_name_var] = vars()[team_name_var].set_index('Tm')
        vars()[team_name_var] = vars()[team_name_var].drop(columns = ('Team Int'))
        team_count += 1
    count = 0
    for name in team_names_str:
        team_name_var =  'Schedule_Lineups_' + team_names_str[count]
        vars()[team_name_var] = pd.concat([team_schedule[count],team_lineups[count]], axis = 1, join = 'inner')
        vars()[team_name_var] = vars()[team_name_var].drop(columns = ['1st','2nd','3rd','4th','5th','6th','7th','8th','9th'])
        vars()[team_name_var] = vars()[team_name_var].reset_index()
        vars()[team_name_var] = vars()[team_name_var].groupby('Game Date').get_group(todays_date[date_count_1])
        vars()[team_name_var]["Win Loss"] = vars()[team_name_var]["Win Loss"].apply(lambda x: win_loss[x])
        vars()[team_name_var] = vars()[team_name_var].set_index('Tm')
        vars()[team_name_var].columns = ['Game Date','Opponent','Win Loss']
        count += 1
    team_stats_list = []
    count = 0
    for name in team_names_str:
        team_name_var =  'Schedule_Lineups_' + team_names_str[count]
        team_var = team_names_str[count] + '_Team'
        vars()[team_names_str[count]] = pd.concat([vars()[team_name_var], vars()[team_var]],axis = 1, join = 'inner').reset_index()  
        team_stats_list.append(vars()[team_names_str[count]])
        count += 1
    count = 0
    opponent_list = []
    for item in team_stats_list:
        opponent_list.append(team_stats_list[count]['Opponent'][0])
        count += 1
    opponent_df = pd.DataFrame(opponent_list)
    team_names_df = pd.DataFrame(team_names_str)
    count_df = pd.concat([team_names_df, opponent_df], axis = 1)
    count_df.columns = ['Tm', 'Opponent']
    count = 0
    count_list_1 = []
    count_list_2 = []
    count_df
    for name in team_names_str:
        team_name = count_df['Tm'][count]
        for x in range(count):
            if count_df['Opponent'][x] == team_name:
                break
            else:
                None
        if count < x:
            count_list_1.append(count)
        else:
            count_list_2.append(count)
        count += 1
    stats_list = teams_batter.drop(columns = ['Name','Tm']).columns.to_list()
    stats_list_opp = []
    stats_list_1 = []
    stats_count = 0
    for stat in stats_list:

        stats_list_opp.append(stats_list[stats_count] + '_Pitch_OPP')
        stats_list_1.append(stats_list[stats_count] + '_Pitch')
        stats_count += 1
    for count in count_list_1:
        # Initialize count for stats
        stats_count = 0
        # Establish team name variables to call in previous df's
        team_name_var = team_names_str[count]
        # Create a new variable concatinated df name
        concat_name = team_names_str[count] + '_Concat'
        # Create a new opponent vaiable name
        opp_name_var = opponent_list[count]
        # Set index as 'Opponent' for team name variable
        # Create a new variable name for opponent
        new_opp = opponent_list[count] + '_Opp'
        # print(new_opp)
        # Copy over old dataframe to new one so as to not alter the original
        vars()[new_opp] = (vars()[opp_name_var]).copy()
        # Set new df index to 'Opponent'
        vars()[new_opp] = vars()[opp_name_var].set_index('Opponent')
        vars()[team_name_var] = vars()[team_name_var].reset_index().set_index('Tm')
        vars()[new_opp].columns = ['Tm','Game Date_opp','Win Loss_opp','G_Pitch_OPP', 'GS_Pitch_OPP', 'IP_Pitch_OPP', 'H_Pitch_OPP', 'R_Pitch_OPP', 'ER_Pitch_OPP', 'BB_Pitch_OPP', 'SO_Pitch_OPP', 'HR_Pitch_OPP', 'HBP_Pitch_OPP', 'ERA_Pitch_OPP', 'AB_Pitch_OPP','2B_Pitch_OPP', '3B_Pitch_OPP', 'IBB_Pitch_OPP', 'GDP_Pitch_OPP', 'SF_Pitch_OPP', 'SB_Pitch_OPP', 'CS_Pitch_OPP', 'PO_Pitch_OPP', 'BF_Pitch_OPP', 'Pit_Pitch_OPP', 'Str_Pitch_OPP', 'StL_Pitch_OPP', 'StS_Pitch_OPP', 'GB/FB_Pitch_OPP', 'LD_Pitch_OPP', 'PU_Pitch_OPP', 'WHIP_Pitch_OPP', 'BAbip_Pitch_OPP', 'SO9_Pitch_OPP','SO/W_Pitch_OPP', 'Players_Pitch_OPP']
        vars()[concat_name] = pd.concat([vars()[team_name_var], vars()[new_opp]], axis = 1, join = 'inner')
        for stat in stats_list_1:
            vars()[concat_name][stats_list_1[stats_count]] = vars()[concat_name][stats_list_1[stats_count]] - vars()[concat_name][stats_list_opp[stats_count]]
            stats_count += 1
        vars()[concat_name] = vars()[concat_name].drop(columns = stats_list_opp)
        vars()[concat_name] = vars()[concat_name].reset_index()
        vars()[concat_name] = vars()[concat_name].drop(columns = 'index')
        vars()[concat_name].columns = ['Tm','Game Date','Opponent','Win Loss', 'G_Pitch','GS_Pitch', 'IP_Pitch', 'H_Pitch', 'R_Pitch', 'ER_Pitch', 'BB_Pitch','SO_Pitch', 'HR_Pitch', 'HBP_Pitch', 'ERA_Pitch', 'AB_Pitch','2B_Pitch', '3B_Pitch', 'IBB_Pitch', 'GDP_Pitch', 'SF_Pitch','SB_Pitch', 'CS_Pitch', 'PO_Pitch', 'BF_Pitch', 'Pit_Pitch','Str_Pitch', 'StL_Pitch', 'StS_Pitch', 'GB/FB_Pitch', 'LD_Pitch','PU_Pitch', 'WHIP_Pitch', 'BAbip_Pitch', 'SO9_Pitch', 'SO/W_Pitch','Players_Pitch', 'Tm_opp', 'Game Date_opp', 'Win Loss_opp','Players_Pitch_OPP']
        vars()[concat_name] = vars()[concat_name].drop(columns = ['Tm_opp', 'Game Date_opp', 'Win Loss_opp','Players_Pitch_OPP', 'SO/W_Pitch', 'IP_Pitch', 'Players_Pitch'])
        vars()[concat_name] = vars()[concat_name].set_index('Opponent')
        series_list_4.append(vars()[concat_name])
    for count in count_list_2:
        # Initialize a count for stats
        stats_count = 0
        # Pull team var names to call in old df's
        team_name_var = team_names_str[count]

        vars()[team_name_var] = vars()[team_name_var].set_index('Opponent')
        # vars()[team_name_var] = vars()[concat_name].set_index('Opponent')
        # Create a new concat variable name
        concat_name = team_names_str[count] + '_Concat'
        # Create an opponent variable name
        opp_name_var = opponent_list[count]
        # Create a new opp name to not disturb old df
        new_opp = opponent_list[count] + '_Opp'
        # Copy old df to new variable name
        vars()[new_opp] = (vars()[opp_name_var]).copy()
        vars()[new_opp] = vars()[new_opp].reset_index()
        if len(vars()[new_opp].columns) == 38:
            vars()[new_opp] = vars()[new_opp].drop(columns = 'index')
        else:
            pass
        vars()[new_opp].columns = ['Opponent','Game Date_opp','Tm','Win Loss_opp','G_Pitch_OPP', 'GS_Pitch_OPP', 'IP_Pitch_OPP', 'H_Pitch_OPP', 'R_Pitch_OPP', 'ER_Pitch_OPP', 'BB_Pitch_OPP', 'SO_Pitch_OPP', 'HR_Pitch_OPP', 'HBP_Pitch_OPP', 'ERA_Pitch_OPP', 'AB_Pitch_OPP', '2B_Pitch_OPP', '3B_Pitch_OPP', 'IBB_Pitch_OPP', 'GDP_Pitch_OPP', 'SF_Pitch_OPP', 'SB_Pitch_OPP', 'CS_Pitch_OPP', 'PO_Pitch_OPP', 'BF_Pitch_OPP', 'Pit_Pitch_OPP', 'Str_Pitch_OPP', 'StL_Pitch_OPP', 'StS_Pitch_OPP', 'GB/FB_Pitch_OPP', 'LD_Pitch_OPP', 'PU_Pitch_OPP', 'WHIP_Pitch_OPP', 'BAbip_Pitch_OPP', 'SO9_Pitch_OPP', 'SO/W_Pitch_OPP', 'Opponent Int']
        vars()[new_opp] = vars()[new_opp].set_index('Opponent')
        vars()[concat_name] = pd.concat([vars()[team_name_var], vars()[new_opp]], axis = 1, join = 'inner')
        for stat in stats_list:
            vars()[concat_name][stats_list_1[stats_count]] = vars()[concat_name][stats_list_1[stats_count]] - vars()[concat_name][stats_list_opp[stats_count]]
            stats_count += 1
        vars()[concat_name] = vars()[concat_name].drop(columns = ['G_Pitch_OPP', 'GS_Pitch_OPP', 'IP_Pitch_OPP', 'H_Pitch_OPP', 'R_Pitch_OPP', 'ER_Pitch_OPP', 'BB_Pitch_OPP', 'SO_Pitch_OPP', 'HR_Pitch_OPP', 'HBP_Pitch_OPP', 'ERA_Pitch_OPP', 'AB_Pitch_OPP', '2B_Pitch_OPP', '3B_Pitch_OPP', 'IBB_Pitch_OPP', 'GDP_Pitch_OPP', 'SF_Pitch_OPP', 'SB_Pitch_OPP', 'CS_Pitch_OPP', 'PO_Pitch_OPP', 'BF_Pitch_OPP', 'Pit_Pitch_OPP', 'Str_Pitch_OPP', 'StL_Pitch_OPP', 'StS_Pitch_OPP', 'GB/FB_Pitch_OPP', 'LD_Pitch_OPP', 'PU_Pitch_OPP', 'WHIP_Pitch_OPP', 'BAbip_Pitch_OPP', 'SO9_Pitch_OPP', 'SO/W_Pitch_OPP', 'Opponent Int', 'SO/W_Pitch','Game Date_opp','Win Loss_opp','Players_Pitch','IP_Pitch'])
        vars()[concat_name] = vars()[concat_name].reset_index().set_index('Opponent')
        vars()[concat_name].drop(vars()[concat_name].columns[len(vars()[concat_name].columns)-1], axis=1, inplace=True)
        vars()[concat_name]['Tm'] = team_name_var

        series_list_3.append(vars()[concat_name])
    print(date_count_1)
    date_count_1 += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177


In [7]:
todays_date[date_count_1 - 1]

'2019-09-28'

In [8]:
date_count_1

178

In [9]:
first = pd.concat(series_list_1)
second = pd.concat(series_list_2)
third = pd.concat(series_list_3)
fourth = pd.concat(series_list_4)
second.tail()

,Tm,Game Date,Win Loss,G,PA,AB,R,H,2B,3B,...,HBP,SH,SF,GDP,SB,CS,BA,OBP,SLG,OPS
Opponent,,,,,,,,,,,,,,,,,,,,,
As,Angels,2019-09-24,1,1.0,-0.027244,-0.049679,3.961538,6.057692,2.000000,0.423077,...,1.153846,0.461538,0.211538,0.192308,0.442308,0.000000,0.039340,0.031556,0.007266,0.038822
As,Angels,2019-09-25,0,1.0,-0.084772,-0.158242,2.788069,3.830455,1.348823,0.872841,...,0.226688,0.215385,0.226688,0.657457,0.430769,0.215385,0.036953,0.052674,0.131745,0.184418
Astros,Angels,2019-09-26,1,1.0,1.791005,1.558201,2.394180,2.814815,0.695767,0.185185,...,0.185185,0.000000,0.185185,-0.153439,0.185185,0.370370,-0.005021,0.014309,-0.202259,-0.187950
Astros,Angels,2019-09-27,0,1.0,-0.029762,0.007275,0.686508,0.801587,0.392857,0.265873,...,-1.027778,0.000000,0.222222,-0.226190,-0.047619,0.043651,-0.025870,-0.015792,-0.081232,-0.097024
Astros,Angels,2019-09-28,0,1.0,0.035088,-0.035088,-0.403509,-1.771930,-0.017544,0.210526,...,-0.140351,0.017544,0.385965,-0.122807,-0.298246,0.017544,-0.069036,-0.038193,-0.183704,-0.221897


In [10]:
first.tail()

,Game Date,Win Loss,G,PA,AB,R,H,2B,3B,HR,...,SH,SF,GDP,SB,CS,BA,OBP,SLG,OPS,Tm
Opponent,,,,,,,,,,,,,,,,,,,,,
Reds,2019-09-28,0,1.0,0.385572,0.198779,1.111940,-1.365672,-0.298507,-0.208955,0.621891,...,-0.042289,-0.251244,-0.044776,-1.253731,-0.208955,0.011470,0.040090,0.088397,0.128487,Pirates
Yankees,2019-09-28,1,1.0,-0.200674,0.102328,-5.227941,-4.852941,-0.838235,0.176471,-1.720588,...,0.341912,-0.595588,0.694853,-0.562500,0.000000,-0.086509,-0.128406,-0.259962,-0.388368,Rangers
Twins,2019-09-28,0,1.0,-0.147324,-0.270005,-0.378378,2.070747,-0.006359,0.527027,-0.186010,...,0.000000,0.173291,-0.004769,-0.003180,0.175676,-0.016568,0.012945,-0.090587,-0.077642,Royals
White_Sox,2019-09-28,0,1.0,-0.663448,-0.557931,-0.759310,-1.662759,-0.995172,-0.206897,-1.356552,...,0.000000,-0.413793,0.026207,-0.620690,0.660000,0.015035,0.012759,-0.133487,-0.120727,Tigers
White_Sox,2019-09-28,1,1.0,-0.663448,-0.557931,-0.759310,-1.662759,-0.995172,-0.206897,-1.356552,...,0.000000,-0.413793,0.026207,-0.620690,0.660000,0.015035,0.012759,-0.133487,-0.120727,Tigers


In [11]:
first = first.reset_index().set_index(['Opponent','Tm','Game Date','Win Loss'])
third = third.reset_index().set_index(['Opponent','Tm','Game Date','Win Loss'])
first_df = pd.merge(first, third, on = ['Opponent', 'Tm','Game Date','Win Loss'])
first_df = first_df.reset_index()
second = second.reset_index().set_index(['Opponent','Tm','Game Date','Win Loss'])
fourth = fourth.reset_index().set_index(['Opponent','Tm','Game Date','Win Loss'])
second_df = pd.merge(second, fourth, on = ['Opponent', 'Tm','Game Date','Win Loss'])
second_df = second_df.reset_index()
data = pd.concat([first_df, second_df], axis = 0, join = 'inner')
second_df.head()

,Opponent,Tm,Game Date,Win Loss,G,PA,AB,R,H,2B,...,Pit_Pitch,Str_Pitch,StL_Pitch,StS_Pitch,GB/FB_Pitch,LD_Pitch,PU_Pitch,WHIP_Pitch,BAbip_Pitch,SO9_Pitch
0,Rangers,Angels,2019-04-04,0,1.0,0.347953,-0.060429,-1.893762,-3.224172,-1.181287,...,-219,-1.61,-0.48,-0.17,-0.58,-0.76,0.17,0.831582,-0.161382,-26.5
1,Rangers,Angels,2019-04-05,1,1.0,0.808824,0.828976,2.459695,3.751634,1.725490,...,-3,-0.73,-0.39,-0.19,-1.19,0.10,-0.01,1.169908,0.053750,-35.2
2,Rangers,Angels,2019-04-06,1,1.0,0.091101,0.543891,-0.818100,2.316742,0.582805,...,127,1.37,0.12,-0.04,0.75,1.26,0.21,0.632419,0.096139,4.5
3,Rangers,Angels,2019-04-07,1,1.0,-0.073375,0.459119,-0.237596,2.830189,0.429769,...,-147,-1.28,-0.41,-0.36,-0.72,-0.82,-0.09,-0.430556,-0.037372,-29.5
4,Brewers,Angels,2019-04-08,1,1.0,0.823846,0.449715,3.164137,2.438330,0.701139,...,33,0.51,0.12,0.11,0.87,-0.37,-0.10,0.717734,0.106485,21.4


In [12]:
data = data.replace([np.inf], 100)
data = data.replace([np.NaN], 0)
data = data.replace([-np.inf], -100)
data = data.set_index(['Game Date','Opponent','Tm','Win Loss'])
data = data.sort_values('Game Date')

In [14]:
Final_Game_Data = data
Final_Game_Data.to_csv(r'../Baseball/Final_Game_Data_2018.csv')

In [ ]:
# Run random forest to determine the stats that contribute most to a win
 # Define features set
X = data.copy()
X = X.drop(columns = ['G', 'G_Pitch'])
y = data.reset_index()['Win Loss']
# Splitting into Train and Test sets
accuracy_list = []
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = x)
    X_train = X[-120:-30]
    X_test = X[-30:-15]
    y_train = y[-120:-30]
    y_test = y[-30:-15]
    # Create a random forest classifier
    rf_model = RandomForestClassifier(n_estimators=8000, random_state = x)
    # Fitting the model
    rf_model = rf_model.fit(X_train, y_train)
    # Making predictions using the testing data
    predictions = rf_model.predict(X_test)
    # Calculating the confusion matrix
    cm = confusion_matrix(y_test, predictions)
    cm_df = pd.DataFrame(
        cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
    )

    # Calculating the accuracy score
    acc_score = accuracy_score(y_test, predictions)
    print(acc_score)
    accuracy_list.append(acc_score)
# Displaying results
print(accuracy_list)
# Random Forests in sklearn will automatically calculate feature importance
importances = rf_model.feature_importances_
# We can sort the features by their importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

0.5333333333333333
0.5333333333333333
0.4666666666666667
0.5333333333333333
0.5333333333333333
0.5333333333333333
0.5333333333333333
0.4666666666666667
0.5333333333333333
0.5333333333333333
0.5333333333333333
0.5333333333333333
0.5333333333333333
0.5333333333333333
0.5333333333333333
0.5333333333333333
0.5333333333333333
0.5333333333333333
0.5333333333333333
0.5333333333333333
0.5333333333333333
0.5333333333333333


In [21]:
X_test['Prediction'] = predictions
X_test.reset_index()[['Tm','Opponent','Game Date','Win Loss','Prediction']]

C:\Users\taylo\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Tm,Opponent,Game Date,Win Loss,Prediction
0,Braves,Mets,2019-09-28,0,0
1,Brewers,Rockies,2019-09-28,0,1
2,Cards,Cubs,2019-09-28,0,1
3,D_Backs,Padres,2019-09-28,1,1
4,Dodgers,Giants,2019-09-28,1,1
5,Indians,Nats,2019-09-28,0,1
6,Jays,Rays,2019-09-28,1,1
7,Marlins,Phillies,2019-09-28,0,1
8,Orioles,Red_Sox,2019-09-28,1,1
9,Pirates,Reds,2019-09-28,0,0
